# 3D forest height mapping

In [2]:
libs <- c(
  "tidyverse", "sf", "geodata","terra", "classInt", "rayshader")

In [ ]:
installed_libs <- libs %in% rownames(
  installed.packages()
)

In [ ]:
if(any(installed_libs == F)){
  install.packages(
    libs[!installed_libs])
}
invisible(lapply(
  libs,
  library,
  character.only = T
))